In [1]:
# Para trabajar con regex
# -----------------------------------------------------------------------
import re

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os


# Para trabajar con los DataFrames
# -----------------------------------------------------------------------
import pandas as pd

# Para poner barras progreso
# -----------------------------------------------------------------------
from tqdm import tqdm



# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Ignorar warings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# Librerias para generar gráficas
# ----------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


Procesamos el archivo con información provincial y autonómica de renta media por hogar extraído del INE

In [2]:
df_renta_neta_hogar_comunidades_provincias = pd.read_csv("datos/ine/renta_media_comunidades_provincias.csv", encoding= 'latin-1', sep=';')

In [3]:
df_renta_neta_hogar_comunidades_provincias.head()

,Comunidades y Ciudades Autónomas,Provincias,Islas,Indicadores de renta media,Periodo,Total
0,Andalucía,NaN,NaN,Renta neta media por hogar,2022,29.994
1,Andalucía,Almería,NaN,Renta neta media por hogar,2022,28.442
2,Andalucía,Cádiz,NaN,Renta neta media por hogar,2022,30.055
3,Andalucía,Córdoba,NaN,Renta neta media por hogar,2022,29.082
4,Andalucía,Granada,NaN,Renta neta media por hogar,2022,29.113


Eliminamos las columnas "Indicadores de renta media", "Islas" y "Periodo".

In [4]:
# Eliminar las columnas "Indicadores de renta media", "Islas" y "Periodo"
df_renta_neta_hogar_comunidades_provincias.drop(columns=['Indicadores de renta media','Islas','Periodo'], inplace=True)

# Mostrar las primeras filas del DataFrame actualizado
df_renta_neta_hogar_comunidades_provincias.head()

,Comunidades y Ciudades Autónomas,Provincias,Total
0,Andalucía,NaN,29.994
1,Andalucía,Almería,28.442
2,Andalucía,Cádiz,30.055
3,Andalucía,Córdoba,29.082
4,Andalucía,Granada,29.113


In [5]:
df_renta_neta_hogar_comunidades_provincias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 3 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Comunidades y Ciudades Autónomas  71 non-null     object 
 1   Provincias                        52 non-null     object 
 2   Total                             71 non-null     float64
dtypes: float64(1), object(2)
memory usage: 1.8+ KB


In [6]:
# Convertimos la columna "Total" a int64
df_renta_neta_hogar_comunidades_provincias['Total'] = df_renta_neta_hogar_comunidades_provincias['Total'].astype(str)
df_renta_neta_hogar_comunidades_provincias['Total'] = df_renta_neta_hogar_comunidades_provincias['Total'].str.replace('.', '').astype('int64')
df_renta_neta_hogar_comunidades_provincias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 3 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Comunidades y Ciudades Autónomas  71 non-null     object
 1   Provincias                        52 non-null     object
 2   Total                             71 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.8+ KB


Vamos a separar los datos en dos dataframes uno para las Comunidades autónomas y otro para las provincias.

In [7]:
# Filtramos donde "Comunidades y Ciudades Autónomas" no sea NaN y "Provincias" sea NaN
df_renta_neta_hogar_comunidades = df_renta_neta_hogar_comunidades_provincias[df_renta_neta_hogar_comunidades_provincias['Comunidades y Ciudades Autónomas'].notna() & df_renta_neta_hogar_comunidades_provincias['Provincias'].isna()].copy().reset_index(drop=True)

# Filtramos donde "Provincias" no sea NaN
df_renta_neta_hogar_provincias = df_renta_neta_hogar_comunidades_provincias[df_renta_neta_hogar_comunidades_provincias['Provincias'].notna()].copy().reset_index(drop=True)

In [8]:
df_renta_neta_hogar_comunidades.head()

,Comunidades y Ciudades Autónomas,Provincias,Total
0,Andalucía,NaN,29994
1,Aragón,NaN,35569
2,"Asturias, Principado de",NaN,33242
3,"Balears, Illes",NaN,40597
4,Canarias,NaN,33274


In [9]:
df_renta_neta_hogar_provincias.head()  

,Comunidades y Ciudades Autónomas,Provincias,Total
0,Andalucía,Almería,28442
1,Andalucía,Cádiz,30055
2,Andalucía,Córdoba,29082
3,Andalucía,Granada,29113
4,Andalucía,Huelva,29133


## Procesamos primero el dataframe de Comunidades autónomas.

En primer lugar eliminamos la columna de "Provincias"

In [10]:
# Eliminar la columna "Provincias" del DataFrame "df_renta_neta_hogar_comunidades"
df_renta_neta_hogar_comunidades.drop(columns=['Provincias'], inplace=True)

# Mostrar las primeras filas del DataFrame actualizado
df_renta_neta_hogar_comunidades.head()

,Comunidades y Ciudades Autónomas,Total
0,Andalucía,29994
1,Aragón,35569
2,"Asturias, Principado de",33242
3,"Balears, Illes",40597
4,Canarias,33274


In [11]:
df_renta_neta_hogar_comunidades.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Comunidades y Ciudades Autónomas  19 non-null     object
 1   Total                             19 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 436.0+ bytes


Renombramos las columnas

In [12]:
df_renta_neta_hogar_comunidades.columns = ["Nombre comunidad", "Renta media por hogar en 2022"]

In [13]:
df_renta_neta_hogar_comunidades.head()

,Nombre comunidad,Renta media por hogar en 2022
0,Andalucía,29994
1,Aragón,35569
2,"Asturias, Principado de",33242
3,"Balears, Illes",40597
4,Canarias,33274


Falta añadir el Código de Comunidad, que puede servir de id de cara a futuras relaciones entre tablas, vamos a copiarlo de la columna de codigo del dataframe de datos autonómicos del censo.

In [14]:
# Leer el fichero que contiene el censo de 2022 por comunidades autónomas
df_censo_2022_comunidades=pd.read_csv("datos_limpiados/comunidades/censo_2022_autonomico_limpio.csv")

In [15]:
df_censo_2022_comunidades.head()

,Código comunidad,Nombre comunidad,Total
0,1,Andalucía,8511167
1,2,Aragón,1328215
2,3,"Asturias, Principado de",1004960
3,4,"Balears, Illes",1187043
4,5,Canarias,2185607


In [16]:
# Copiar la columna 'Código comunidad' del dataframe de censo al de renta
df_renta_neta_hogar_comunidades['Código comunidad'] = df_censo_2022_comunidades['Código comunidad']

# Mostrar las primeras filas del DataFrame actualizado
df_renta_neta_hogar_comunidades.head()


,Nombre comunidad,Renta media por hogar en 2022,Código comunidad
0,Andalucía,29994,1
1,Aragón,35569,2
2,"Asturias, Principado de",33242,3
3,"Balears, Illes",40597,4
4,Canarias,33274,5


Reordenamos las columnas del dataframe

In [17]:
# Reordenar las columnas para que el codigo y nombre de comunidad aparezcan primero
columnas_prioritarias = ['Código comunidad', 'Nombre comunidad']
columnas_ordenadas = columnas_prioritarias + [col for col in df_renta_neta_hogar_comunidades.columns if col not in columnas_prioritarias]
df_renta_neta_hogar_comunidades = df_renta_neta_hogar_comunidades[columnas_ordenadas]

In [18]:
df_renta_neta_hogar_comunidades.head()

,Código comunidad,Nombre comunidad,Renta media por hogar en 2022
0,1,Andalucía,29994
1,2,Aragón,35569
2,3,"Asturias, Principado de",33242
3,4,"Balears, Illes",40597
4,5,Canarias,33274


In [19]:
df_renta_neta_hogar_comunidades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 3 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Código comunidad               19 non-null     int64 
 1   Nombre comunidad               19 non-null     object
 2   Renta media por hogar en 2022  19 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 588.0+ bytes


In [20]:
# Guardamos el DataFrame en un archivo CSV
df_renta_neta_hogar_comunidades.to_csv("datos_limpiados/comunidades/renta_neta_hogar_comunidades_limpio.csv", index=False)

## Limpiamos ahora el dataframe con la información de renta por provincias

In [21]:
df_renta_neta_hogar_provincias.head()

,Comunidades y Ciudades Autónomas,Provincias,Total
0,Andalucía,Almería,28442
1,Andalucía,Cádiz,30055
2,Andalucía,Córdoba,29082
3,Andalucía,Granada,29113
4,Andalucía,Huelva,29133


Renombramos las columnas

In [22]:
df_renta_neta_hogar_provincias.columns = ["Nombre comunidad", "Nombre provincia", "Renta media por hogar en 2022"]

El dataframe está bastante limpio. Solo falta añadir columnas de código para provincias y comunidades. Ambas las copiaremos del dataframe de censo provincial

In [23]:
# Leer el fichero que contiene el censo de 2022 por comunidades autónomas
df_censo_2022_provincias=pd.read_csv("datos_limpiados/provincias/censo_2022_provincias_limpio.csv")

In [24]:
df_censo_2022_provincias.head()

,Código comunidad,Nombre comunidad,Código provincia,Nombre provincia,Total
0,1,Andalucía,29,Málaga,1715109
1,1,Andalucía,23,Jaén,624191
2,1,Andalucía,4,Almería,739541
3,1,Andalucía,21,Huelva,531094
4,1,Andalucía,11,Cádiz,1250545


Para el codigo de provinicas, lo primero que vamos a ahcer es reordenar la colmna de "Nombre provincia" del dataframe de renta neta para que este en el mismo orden que el dataframe de censo.

In [25]:
# Reordenar df_renta_neta_hogar_provincias según el orden de "Nombre provincia" en df_censo_2022_provincias
df_renta_neta_hogar_provincias = df_renta_neta_hogar_provincias.set_index("Nombre provincia").loc[df_censo_2022_provincias["Nombre provincia"]].reset_index()

# Mostrar las primeras filas del DataFrame ordenado
df_renta_neta_hogar_provincias


,Nombre provincia,Nombre comunidad,Renta media por hogar en 2022
0,Málaga,Andalucía,30644
1,Jaén,Andalucía,27429
2,Almería,Andalucía,28442
3,Huelva,Andalucía,29133
4,Cádiz,Andalucía,30055
5,Sevilla,Andalucía,31899
6,Granada,Andalucía,29113
7,Córdoba,Andalucía,29082
8,Zaragoza,Aragón,36098
9,Teruel,Aragón,32889


Ahora ya, podemos proceder a copiar 

In [26]:
df_renta_neta_hogar_provincias['Código provincia'] = df_censo_2022_provincias['Código provincia']

Ahora, copiamos la columna de Código Comunidad del dataframe de censo en el dataframe de renta neta

In [27]:
df_renta_neta_hogar_provincias['Código comunidad'] = df_censo_2022_provincias['Código comunidad']

In [28]:
# Reordenar las columnas para que los códigos y nombres de comunidad y provincia aparezcan primero
columnas_prioritarias = ['Código comunidad', 'Nombre comunidad', 'Código provincia', 'Nombre provincia']
columnas_ordenadas = columnas_prioritarias + [col for col in df_renta_neta_hogar_provincias.columns if col not in columnas_prioritarias]
df_renta_neta_hogar_provincias = df_renta_neta_hogar_provincias[columnas_ordenadas]

In [29]:
df_renta_neta_hogar_provincias.head()

,Código comunidad,Nombre comunidad,Código provincia,Nombre provincia,Renta media por hogar en 2022
0,1,Andalucía,29,Málaga,30644
1,1,Andalucía,23,Jaén,27429
2,1,Andalucía,4,Almería,28442
3,1,Andalucía,21,Huelva,29133
4,1,Andalucía,11,Cádiz,30055


In [30]:
df_renta_neta_hogar_provincias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Código comunidad               52 non-null     int64 
 1   Nombre comunidad               52 non-null     object
 2   Código provincia               52 non-null     int64 
 3   Nombre provincia               52 non-null     object
 4   Renta media por hogar en 2022  52 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 2.2+ KB


In [31]:
# Guardamos el DataFrame en un archivo CSV
df_renta_neta_hogar_provincias.to_csv("datos_limpiados/provincias/renta_neta_hogar_comunidades_provincias.csv", index=False)

## Limpiamos los datos de renta media por hogar de los municipios

Estos datos, debido a la magnitud del archivo del INE no pueden ser procesados como csv, por ello, se han guaradado en una Hoja de Cálculo de Excel y requieren un tratamiento diferente en Python para limpiarlos. Además, también debido al tamaño del archivo, no se pudo filtrar en la descarga del INE los datos que itneresan a nuestro análisis, así que hay que seleccionar y filtrar los datos para quedarnos solo con la información de renta media por municipios.

In [32]:
# Cargar el archivo Excel y mostrar las hojas disponibles
xls = pd.ExcelFile("datos/ine/renta_media_municipio.xlsx")
xls.sheet_names

['tabla-30824']

In [33]:
# Cargar la hoja específica en un DataFrame
df_renta_media_hogar_clean = pd.read_excel(xls, sheet_name='tabla-30824', skiprows=4)

# Renombrar las columnas si es necesario (por ahora dejando los nombres detectados)
df_renta_media_hogar_clean = df_renta_media_hogar_clean.dropna(how='all', axis=1)  # Eliminar columnas completamente vacías
df_renta_media_hogar_clean = df_renta_media_hogar_clean.dropna(how='all')  # Eliminar filas completamente vacías

# Mostrar las primeras filas después de la limpieza
df_renta_media_hogar_clean.head()


,Unidades: €,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48
1,,Renta neta media por persona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Renta neta media por hogar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Media de la renta por unidad de consumo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mediana de la renta por unidad de consumo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Renta bruta media por persona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Renta bruta media por hogar,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,2022,2021,2020,2019,2018,2017,2016,2015,2022,2021,2020,2019,2018,2017,2016,2015,2022,2021,2020,2019,2018,2017,2016,2015,2022,2021,2020,2019,2018,2017,2016,2015,2022,2021,2020,2019,2018,2017,2016,2015,2022,2021,2020,2019,2018,2017,2016,2015
3,01001 Alegría-Dulantzi,15116,14647,13969,14299,13361,13281,13086,12936,38507,37334,36393,36974,34510,34618,34373,33702,22449,21779,20966,,,,,,20650,20650,19950,,,,,,18082,17592,16728,,,,,,46062,44841,43580,,,,,
4,0100101 Alegría-Dulantzi distrito 01,15116,14647,13969,,,,,,38507,37334,36393,,,,,,22449,21779,20966,,,,,,20650,20650,19950,,,,,,18082,17592,16728,,,,,,46062,44841,43580,,,,,
5,0100101001 Alegría-Dulantzi sección 01001,15473,15191,14553,,,,,,37739,37285,36314,,,,,,22640,22357,21486,,,,,,21350,20650,20650,,,,,,18561,18284,17402,,,,,,45270,44875,43424,,,,,


Procesamos el archivo eliminando filas innecesarias y renombrando las columnas correctamente

In [34]:
# Identificar la fila que contiene los nombres correctos de las columnas
column_names = df_renta_media_hogar_clean.iloc[1]  # Segunda fila contiene los nombres de columnas
df_renta_media_hogar_clean = df_renta_media_hogar_clean.iloc[2:]  # Eliminar filas innecesarias

# Asignar nombres de columnas correctos
df_renta_media_hogar_clean.columns = column_names
df_renta_media_hogar_clean = df_renta_media_hogar_clean.reset_index(drop=True)

# Eliminar columnas completamente vacías
df_renta_media_hogar_clean = df_renta_media_hogar_clean.dropna(how='all', axis=1)

# Mostrar las primeras filas del DataFrame limpio
df_renta_media_hogar_clean.head()


2,,2022,2021,2020,2019,2018,2017,2016,2015,2022,2021,2020,2019,2018,2017,2016,2015,2022,2021,2020,2019,2018,2017,2016,2015,2022,2021,2020,2019,2018,2017,2016,2015,2022,2021,2020,2019,2018,2017,2016,2015,2022,2021,2020,2019,2018,2017,2016,2015
0,01001 Alegría-Dulantzi,15116,14647,13969,14299,13361,13281,13086,12936,38507,37334,36393,36974,34510,34618,34373,33702,22449,21779,20966,,,,,,20650,20650,19950,,,,,,18082,17592,16728,,,,,,46062,44841,43580,,,,,
1,0100101 Alegría-Dulantzi distrito 01,15116,14647,13969,,,,,,38507,37334,36393,,,,,,22449,21779,20966,,,,,,20650,20650,19950,,,,,,18082,17592,16728,,,,,,46062,44841,43580,,,,,
2,0100101001 Alegría-Dulantzi sección 01001,15473,15191,14553,,,,,,37739,37285,36314,,,,,,22640,22357,21486,,,,,,21350,20650,20650,,,,,,18561,18284,17402,,,,,,45270,44875,43424,,,,,
3,0100101002 Alegría-Dulantzi sección 01002,14811,14172,13460,,,,,,39221,37380,36467,,,,,,22286,21274,20513,,,,,,19950,19950,18550,,,,,,17673,16989,16140,,,,,,46798,44810,43727,,,,,
4,01002 Amurrio,16011,15192,14683,14892,14103,13862,13691,13800,39221,37410,36362,36839,35037,34411,33936,34421,23589,22451,21797,,,,,,22050,20650,19950,,,,,,19088,18014,17388,,,,,,46758,44361,43061,,,,,


El Dataframe todavía no está limpiado y estructurado correctamente. Seguimos procesándolo

In [35]:
# Renombrar la primera columna a 'Código y Nombre Municipio' para identificarla correctamente
df_renta_media_hogar_clean = df_renta_media_hogar_clean.rename(columns={df_renta_media_hogar_clean.columns[0]: "Código y Nombre Municipio"})

# Eliminar filas donde 'Código y Nombre Municipio' sea NaN
df_renta_media_hogar_clean = df_renta_media_hogar_clean.dropna(subset=["Código y Nombre Municipio"])

# Restablecer el índice después de la limpieza
df_renta_media_hogar_clean = df_renta_media_hogar_clean.reset_index(drop=True)

# Mostrar las primeras filas del DataFrame limpio
df_renta_media_hogar_clean.head()


2,Código y Nombre Municipio,2022,2021,2020,2019,2018,2017,2016,2015,2022,2021,2020,2019,2018,2017,2016,2015,2022,2021,2020,2019,2018,2017,2016,2015,2022,2021,2020,2019,2018,2017,2016,2015,2022,2021,2020,2019,2018,2017,2016,2015,2022,2021,2020,2019,2018,2017,2016,2015
0,01001 Alegría-Dulantzi,15116,14647,13969,14299,13361,13281,13086,12936,38507,37334,36393,36974,34510,34618,34373,33702,22449,21779,20966,,,,,,20650,20650,19950,,,,,,18082,17592,16728,,,,,,46062,44841,43580,,,,,
1,0100101 Alegría-Dulantzi distrito 01,15116,14647,13969,,,,,,38507,37334,36393,,,,,,22449,21779,20966,,,,,,20650,20650,19950,,,,,,18082,17592,16728,,,,,,46062,44841,43580,,,,,
2,0100101001 Alegría-Dulantzi sección 01001,15473,15191,14553,,,,,,37739,37285,36314,,,,,,22640,22357,21486,,,,,,21350,20650,20650,,,,,,18561,18284,17402,,,,,,45270,44875,43424,,,,,
3,0100101002 Alegría-Dulantzi sección 01002,14811,14172,13460,,,,,,39221,37380,36467,,,,,,22286,21274,20513,,,,,,19950,19950,18550,,,,,,17673,16989,16140,,,,,,46798,44810,43727,,,,,
4,01002 Amurrio,16011,15192,14683,14892,14103,13862,13691,13800,39221,37410,36362,36839,35037,34411,33936,34421,23589,22451,21797,,,,,,22050,20650,19950,,,,,,19088,18014,17388,,,,,,46758,44361,43061,,,,,


En este dataframe hay varias métricas de renta mezcladas, vamos a separarlas cada una en un dataframe

In [36]:
# Separar en diferentes DataFrames según el tipo de renta

# Identificar las posiciones de las columnas clave para segmentar
renta_neta_persona_cols = df_renta_media_hogar_clean.columns[:10]  # Aproximadamente las primeras columnas de renta neta por persona
renta_neta_hogar_cols = df_renta_media_hogar_clean.columns[10:20]  # Siguientes columnas de renta neta por hogar
renta_bruta_hogar_cols = df_renta_media_hogar_clean.columns[20:]  # Últimas columnas de renta bruta por hogar

# Crear DataFrames separados
df_renta_neta_persona = df_renta_media_hogar_clean[renta_neta_persona_cols].copy()
df_renta_neta_hogar = df_renta_media_hogar_clean.iloc[:, [0]].join(df_renta_media_hogar_clean[renta_neta_hogar_cols]).copy()
df_renta_bruta_hogar = df_renta_media_hogar_clean.iloc[:, [0]].join(df_renta_media_hogar_clean[renta_bruta_hogar_cols]).copy()

A nosotros nos interesa el dataframe de renta neta por hogar.

In [37]:
# Mostrar las primeras filas del dataframe de renta neta por hogar
df_renta_neta_hogar.head()

2,Código y Nombre Municipio,2021,2021,2021,2021,2021,2021,2020,2020,2020,2020,2020,2020,2019,2019,2019,2019,2019,2019,2018,2018,2018,2018,2018,2018,2017,2017,2017,2017,2017,2017,2016,2016,2016,2016,2016,2016,2015,2015,2015,2015,2015,2015,2022,2022,2022,2022,2022,2022,2021,2021,2021,2021,2021,2021,2020,2020,2020,2020,2020,2020
0,01001 Alegría-Dulantzi,14647,37334,21779,20650,17592,44841,13969,36393,20966,19950,16728,43580,14299,36974,,,,,13361,34510,,,,,13281,34618,,,,,13086,34373,,,,,12936,33702,,,,,15116,38507,22449,20650,18082,46062,14647,37334,21779,20650,17592,44841,13969,36393,20966,19950,16728,43580
1,0100101 Alegría-Dulantzi distrito 01,14647,37334,21779,20650,17592,44841,13969,36393,20966,19950,16728,43580,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,15116,38507,22449,20650,18082,46062,14647,37334,21779,20650,17592,44841,13969,36393,20966,19950,16728,43580
2,0100101001 Alegría-Dulantzi sección 01001,15191,37285,22357,20650,18284,44875,14553,36314,21486,20650,17402,43424,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,15473,37739,22640,21350,18561,45270,15191,37285,22357,20650,18284,44875,14553,36314,21486,20650,17402,43424
3,0100101002 Alegría-Dulantzi sección 01002,14172,37380,21274,19950,16989,44810,13460,36467,20513,18550,16140,43727,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,14811,39221,22286,19950,17673,46798,14172,37380,21274,19950,16989,44810,13460,36467,20513,18550,16140,43727
4,01002 Amurrio,15192,37410,22451,20650,18014,44361,14683,36362,21797,19950,17388,43061,14892,36839,,,,,14103,35037,,,,,13862,34411,,,,,13691,33936,,,,,13800,34421,,,,,16011,39221,23589,22050,19088,46758,15192,37410,22451,20650,18014,44361,14683,36362,21797,19950,17388,43061


La columna con los datos de Renta media por hogar en 2022 es la numero 45 así pues nos quedamso con esta y con la de Código y Nombre de Municipio

In [38]:
df_renta_neta_hogar = df_renta_neta_hogar.iloc[:, [0, 44]].copy()
df_renta_neta_hogar.head()

2,Código y Nombre Municipio,2022
0,01001 Alegría-Dulantzi,38507
1,0100101 Alegría-Dulantzi distrito 01,38507
2,0100101001 Alegría-Dulantzi sección 01001,37739
3,0100101002 Alegría-Dulantzi sección 01002,39221
4,01002 Amurrio,39221


Aquí tenemos datos de municipios, mezclados con los de secciones y distritos. Nos interesan solo los de municipios, así que vamos a filtrar el dataframe para quedarnos solo con los datos que nos interesan

In [39]:
# Filtrar el DataFrame para quedarse solo con los municipios
df_renta_neta_hogar_municipios = df_renta_neta_hogar[
    df_renta_neta_hogar["Código y Nombre Municipio"].str.match(r"^\d{5}\s", na=False)
].copy().reset_index(drop=True)

# Mostrar el DataFrame filtrado
df_renta_neta_hogar_municipios.head()


2,Código y Nombre Municipio,2022
0,01001 Alegría-Dulantzi,38507
1,01002 Amurrio,39221
2,01003 Aramaio,51737
3,01004 Artziniega,38089
4,01006 Armiñón,36433


Renombramos las columnas, para quitar el "2" del encabezado del Indice, y poner el inicador de renta que estamos usanto

In [40]:
# Corregir los nombres de las columnas
df_renta_neta_hogar_municipios.columns = ["Código y Nombre Municipio", "Renta media por hogar en 2022"]

# Mostrar el DataFrame corregido
df_renta_neta_hogar_municipios.head()


,Código y Nombre Municipio,Renta media por hogar en 2022
0,01001 Alegría-Dulantzi,38507
1,01002 Amurrio,39221
2,01003 Aramaio,51737
3,01004 Artziniega,38089
4,01006 Armiñón,36433


In [41]:
df_renta_neta_hogar_municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8138 entries, 0 to 8137
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Código y Nombre Municipio      8138 non-null   object
 1   Renta media por hogar en 2022  8138 non-null   object
dtypes: object(2)
memory usage: 127.3+ KB


Formateamos la columna 'Renta media por hogar en 2022'

Al intentar convertirla a int64 vemos que hay valores que dan problemas. Los identificamos y decidimso que hacer con ellos

In [42]:
df_renta_neta_hogar_municipios["Renta media por hogar en 2022"] = df_renta_neta_hogar_municipios["Renta media por hogar en 2022"].astype(str)

valores_problematicos = df_renta_neta_hogar_municipios[
    ~df_renta_neta_hogar_municipios["Renta media por hogar en 2022"].str.replace(r"[^\d]", "", regex=True).str.isnumeric()
]

valores_problematicos


,Código y Nombre Municipio,Renta media por hogar en 2022
86,02036 Golosalvo,.
100,02050 Montalvos,.
127,02077 Villa de Ves,.
205,03068 Famorca,.
264,03130 Tollos,.
...,...,...
7306,47049 Cervillego de la Cruz,.
7351,47095 Moraleja de las Panaderas,.
7683,49099 Losacio,.
7803,49233 Vegalatrave,.


Hay valores que son puntos. Es decir, no se tiene la información numérica de la Renta media. Soun un total de 74 entre 8138, es decir menos del 1 por ciento. Los imputaremos por la mediana de los valores de la columna y proseguiremos con el formateo a entero.

In [43]:
# Calcular la mediana de la columna (ignorando valores no numéricos)
mediana_renta = pd.to_numeric(
    df_renta_neta_hogar_municipios["Renta media por hogar en 2022"].replace(r"[^\d]", "", regex=True), errors='coerce'
).median()

# Reemplazar los valores problemáticos por la mediana
df_renta_neta_hogar_municipios["Renta media por hogar en 2022"] = df_renta_neta_hogar_municipios[
    "Renta media por hogar en 2022"
].replace(r"[^\d]", "", regex=True).replace("", mediana_renta).astype("int64")

# Mostrar las primeras filas para verificar la conversión
df_renta_neta_hogar_municipios.head()


,Código y Nombre Municipio,Renta media por hogar en 2022
0,01001 Alegría-Dulantzi,38507
1,01002 Amurrio,39221
2,01003 Aramaio,51737
3,01004 Artziniega,38089
4,01006 Armiñón,36433


In [44]:
df_renta_neta_hogar_municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8138 entries, 0 to 8137
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Código y Nombre Municipio      8138 non-null   object
 1   Renta media por hogar en 2022  8138 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 127.3+ KB


Separamos el código y el nombre de los municipios en columnas distintas

In [45]:
df_renta_neta_hogar_municipios[['Código municipio', 'Nombre municipio']] = df_renta_neta_hogar_municipios['Código y Nombre Municipio'].str.split(pat=' ', n=1, expand=True)
df_renta_neta_hogar_municipios.drop(columns=['Código y Nombre Municipio'], inplace=True)

In [46]:
#  Convertir las columnas 'Código municipio' a tipo int64
df_renta_neta_hogar_municipios['Código municipio'] = df_renta_neta_hogar_municipios['Código municipio'].astype('int64')

In [47]:
# Reordenar las columnas para que los códigos y nombres aparezcan primero
columnas_prioritarias = ['Código municipio', 'Nombre municipio']
columnas_ordenadas = columnas_prioritarias + [col for col in df_renta_neta_hogar_municipios.columns if col not in columnas_prioritarias]
df_renta_neta_hogar_municipios = df_renta_neta_hogar_municipios[columnas_ordenadas]

In [48]:
df_renta_neta_hogar_municipios.head()

,Código municipio,Nombre municipio,Renta media por hogar en 2022
0,1001,Alegría-Dulantzi,38507
1,1002,Amurrio,39221
2,1003,Aramaio,51737
3,1004,Artziniega,38089
4,1006,Armiñón,36433


La limpieza del archivo ha terminado guardamos el dataframe como csv.

In [49]:
# Guardamos el DataFrame en un archivo CSV
df_renta_neta_hogar_municipios.to_csv("datos_limpiados/municipios/renta_neta_hogar_municipios_limpio.csv", index=False)